# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7860eae1f0>
├── 'a' --> tensor([[-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021]])
└── 'x' --> <FastTreeValue 0x7f7860e99bb0>
    └── 'c' --> tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                        [-1.0499, -1.8174,  2.7077, -0.0087],
                        [ 0.7009,  0.1523, -0.9446, -2.6643]])

In [4]:
t.a

tensor([[-0.9216, -1.4134,  1.0425],
        [ 0.7309,  0.3713, -1.5021]])

In [5]:
%timeit t.a

63.9 ns ± 0.0253 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7860eae1f0>
├── 'a' --> tensor([[ 0.9937,  0.0089,  0.8804],
│                   [-0.3375,  1.5996,  1.1779]])
└── 'x' --> <FastTreeValue 0x7f7860e99bb0>
    └── 'c' --> tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                        [-1.0499, -1.8174,  2.7077, -0.0087],
                        [ 0.7009,  0.1523, -0.9446, -2.6643]])

In [7]:
%timeit t.a = new_value

64.6 ns ± 0.566 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021]]),
    x: Batch(
           c: tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                      [-1.0499, -1.8174,  2.7077, -0.0087],
                      [ 0.7009,  0.1523, -0.9446, -2.6643]]),
       ),
)

In [10]:
b.a

tensor([[-0.9216, -1.4134,  1.0425],
        [ 0.7309,  0.3713, -1.5021]])

In [11]:
%timeit b.a

52.1 ns ± 0.0472 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.8491, -0.5694,  1.8730],
               [-0.5294, -0.6295,  0.8563]]),
    x: Batch(
           c: tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                      [-1.0499, -1.8174,  2.7077, -0.0087],
                      [ 0.7009,  0.1523, -0.9446, -2.6643]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.418 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

857 ns ± 0.268 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 76.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 3.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 566 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f77b89a9430>
├── 'a' --> tensor([[[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]],
│           
│                   [[-0.9216, -1.4134,  1.0425],
│                    [ 0.7309,  0.3713, -1.5021]]])
└── 'x' --> <FastTreeValue 0x7f7844195e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 54.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7860eb6070>
├── 'a' --> tensor([[-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021],
│                   [-0.9216, -1.4134,  1.0425],
│                   [ 0.7309,  0.3713, -1.5021]])
└── 'x' --> <FastTreeValue 0x7f77b89bcc70>
    └── 'c' --> tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                        [-1.0499, -1.8174,  2.7077, -0.0087],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 92.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 46.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]],
       
               [[-0.9216, -1.4134,  1.0425],
                [ 0.7309,  0.3713, -1.5021]]]),
    x: Batch(
           c: tensor([[[-0.9498, -1.1655, -0.2283,  0.7902],
                       [-1.0499, -1.8174,  2.7077, -0.0087],
                       [ 0.7009,  0.1523, -0.9446, -2.6643]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021],
               [-0.9216, -1.4134,  1.0425],
               [ 0.7309,  0.3713, -1.5021]]),
    x: Batch(
           c: tensor([[-0.9498, -1.1655, -0.2283,  0.7902],
                      [-1.0499, -1.8174,  2.7077, -0.0087],
                      [ 0.7009,  0.1523, -0.9446, -2.6643],
                      [-0.9498, -1.1655, -0.2283,  0.7902],
                      [-1.0499, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 45.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
